<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/debugg/debugging_apis.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Se entrega a mas tardar el martes 14 de Noviembre a las 11:55 pm. Vale 40 puntos de tarea.
Te recomiendo leer todo el ejercicio antes de empezar a ahcer codigo, para que te des una idea de que formato y estructura necesitamos

In [ ]:
import pandas as pd
import requests
from collections import defaultdict
import json

Objetivos del Ejercicio:

    Realizar una solicitud a la API de Pokémon para obtener datos sobre un Pokémon específico.
    Cargar y visualizar los datos en bruto para comprender su estructura.
    Limpiar y organizar los datos para un análisis posterior.
    Guardar los datos limpios en un archivo CSV.

El objetivo es crear un `DataFrame` (tidydata) de varios pokemones que ademas contengan metadata de sus moviemientos.  
Para ello deberas realizar request al `API` de pokemon, entender como vienen los datos y despues combinarlos con los `joins` correspondientes.

# API request

Crea codigo para acceder al API de pokemon y hacer requests

In [ ]:
# Function to get data for a specific Pokémon
def get_pokemon_data(pokemon_name):
    url = f'https://pokeapi.co/api/v2/pokemon/{pokemon_name}'
    response = requests.get(url)

    if response.status_code == 200:
        pokemon_data = response.json()
        return pokemon_data
    else:
        print(f"Failed to retrieve data for {pokemon_name}")
        return None

# Example usage
pokemon_name = 'pikachu'  # Replace with the name of the Pokémon you want to get data for
data = get_pokemon_data(pokemon_name)

if data:
    # Convert data to DataFrame and perform operations as required
    # For example, let's create a DataFrame for the Pokémon's moves
    moves_data = pd.DataFrame(data['moves'])
    print(moves_data.head())  # Visualize the first few rows of the moves data


                                                move  \
0  {'name': 'mega-punch', 'url': 'https://pokeapi...   
1  {'name': 'pay-day', 'url': 'https://pokeapi.co...   
2  {'name': 'thunder-punch', 'url': 'https://poke...   
3  {'name': 'slam', 'url': 'https://pokeapi.co/ap...   
4  {'name': 'double-kick', 'url': 'https://pokeap...   

                               version_group_details  
0  [{'level_learned_at': 0, 'move_learn_method': ...  
1  [{'level_learned_at': 0, 'move_learn_method': ...  
2  [{'level_learned_at': 0, 'move_learn_method': ...  
3  [{'level_learned_at': 20, 'move_learn_method':...  
4  [{'level_learned_at': 9, 'move_learn_method': ...  


Compelta el siguiente snippet para obter los datos del API de pokemon.

## Pokemon Data

Para  el tipo de pokemon debes obtener:
1. `pokemon_id`
2. `pokemon_name`
3. `type`(s) cada pokemon tiene a 1 o dos tipos (no mas). Tienes que guardar ambos  

Guarda los `moves` del pokemon pues los usaremos mas adelante. En especial el `name` y la `url` de cada movimiento apra que podamos iterar despues

In [ ]:
# Elige tu pokemon
pokemon_name = 'lucario'

In [ ]:
# Solicitud para obtener datos de un Pokémon
pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{pokemon_name}'
pokemon_response = requests.get(pokemon_url)
pokemon_data = pokemon_response.json()
pokemon_id = pokemon_data['id']
pokemon_name = pokemon_data['name']

# Extraemos los tipos del Pokémon
types = [t['type']['name'] for t in pokemon_data['types']]

# Guardamos los movimientos del Pokémon
moves = pd.DataFrame([{'move_name': move['move']['name'], 'move_url': move['move']['url']} for move in pokemon_data['moves']])

# Mostramos los datos extraídos
print(f'ID: {pokemon_id}')
print(f'Name: {pokemon_name}')
print(f'Types: {types}')
print(moves)

ID: 448
Name: lucario
Types: ['fighting', 'steel']
         move_name                             move_url
0       mega-punch    https://pokeapi.co/api/v2/move/5/
1        ice-punch    https://pokeapi.co/api/v2/move/8/
2    thunder-punch    https://pokeapi.co/api/v2/move/9/
3     swords-dance   https://pokeapi.co/api/v2/move/14/
4        mega-kick   https://pokeapi.co/api/v2/move/25/
..             ...                                  ...
105     steel-beam  https://pokeapi.co/api/v2/move/796/
106  terrain-pulse  https://pokeapi.co/api/v2/move/805/
107       coaching  https://pokeapi.co/api/v2/move/811/
108     tera-blast  https://pokeapi.co/api/v2/move/851/
109     trailblaze  https://pokeapi.co/api/v2/move/885/

[110 rows x 2 columns]


## Moves Data

Ahora para cada movimiento `move` del pokemon debes obtener los siguientes datos usando la `url`
1. `name`
2. `accuracy`
3. `type`
4. `power`
5. `pp`

In [ ]:
# Solicitud para obtener datos de movimientos de un Pokémon

moves_details=[]
for _, move_row in moves.iterrows():
    # Extraemos el URL del movimiento de la columna 'move_url'
    move_url = move_row['move_url']
    moves_response = requests.get(move_url)
    moves_data = moves_response.json()
    moves_details.append({
    'name': moves_data['name'],
    'accuracy': moves_data.get('accuracy', 'N/A'),  # Algunos movimientos no tienen accuracy
    'type': moves_data['type']['name'],
    'power': moves_data.get('power', 'N/A'),  # Algunos movimientos no tienen potencia
    'pp': moves_data['pp']
            })
moves_df = pd.DataFrame(moves_details)
    # Mostramos el DataFrame
print(moves_df)

              name  accuracy      type  power  pp
0       mega-punch      85.0    normal   80.0  20
1        ice-punch     100.0       ice   75.0  15
2    thunder-punch     100.0  electric   75.0  15
3     swords-dance       NaN    normal    NaN  20
4        mega-kick      75.0    normal  120.0   5
..             ...       ...       ...    ...  ..
105     steel-beam      95.0     steel  140.0   5
106  terrain-pulse     100.0    normal   50.0  10
107       coaching       NaN  fighting    NaN  10
108     tera-blast     100.0    normal   80.0  10
109     trailblaze     100.0     grass   50.0  20

[110 rows x 5 columns]


# API Request para varios pokemons

Ahora que sabes como funciona el `API` realiza las peticiones de los siguientes pokemones respecto a sus caracteristicas
y moviemientos, y guardalas en una lista, diciconario o tabla (lo que mas te acomode).

In [ ]:
pokemons =['lucario', 'pikachu', 'gengar', 'charizard', 'blastoise', 'venusaur', 'dragonite', 'mew', 'mewtwo', 'garchomp']

In [ ]:
# Solicitudes para obtener datos de un Pokémon
pokemon_details=[]
for pokemon in pokemons:
    # Datos del Pokémon
        pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{pokemon}'
        response = requests.get(pokemon_url)
        data = response.json()
        # Extraemos los datos relevantes
        pokemon_id = data['id']
        pokemon_name = data['name']
        types = [t['type']['name'] for t in data['types']]
        moves = [{'move_name': move['move']['name'], 'move_url': move['move']['url']} for move in data['moves']]

        # Agregamos los datos al diccionario
        pokemon_details.append({
            'id': pokemon_id,
            'name': pokemon_name,
            'types': types,
            'moves': moves
        })

pokemon_df = pd.DataFrame(pokemon_details)
print(pokemon_df)

    # No olvides incluir los movimientos

    id       name              types  \
0  448    lucario  [fighting, steel]   
1   25    pikachu         [electric]   
2   94     gengar    [ghost, poison]   
3    6  charizard     [fire, flying]   
4    9  blastoise            [water]   
5    3   venusaur    [grass, poison]   
6  149  dragonite   [dragon, flying]   
7  151        mew          [psychic]   
8  150     mewtwo          [psychic]   
9  445   garchomp   [dragon, ground]   

                                               moves  
0  [{'move_name': 'mega-punch', 'move_url': 'http...  
1  [{'move_name': 'mega-punch', 'move_url': 'http...  
2  [{'move_name': 'mega-punch', 'move_url': 'http...  
3  [{'move_name': 'mega-punch', 'move_url': 'http...  
4  [{'move_name': 'mega-punch', 'move_url': 'http...  
5  [{'move_name': 'swords-dance', 'move_url': 'ht...  
6  [{'move_name': 'mega-punch', 'move_url': 'http...  
7  [{'move_name': 'pound', 'move_url': 'https://p...  
8  [{'move_name': 'mega-punch', 'move_url': 'http...  
9  [{'mov

# Evolucion a  Tabular

Ahora para cada request (`pokemon` y `movimiento`) convierte su informacion a datos tabulares. Toma en cuenta que vamos a realizar operaciones sobre esos DataFrames, por lo tanto tienes que elegir un formato adecuado.  
Para cada pokemon, y para cada movimiento debe existir un `DataFrame`

In [ ]:
# Ya los tenía en modo tabular.

# Union de Informacion

1. Obten el nombre y caracteristicas de todos lo movimientos que aparecen en por lo menos 3 pokemons de los de la lista, y guarda los datos de movimiento en un solo `DataFrame`

In [ ]:
# Crearemos un diccionario para contar la frecuencia de cada movimiento entre todos los Pokémon
move_counts = defaultdict(int)

# Recorremos cada lista de movimientos de los Pokémon y contamos los movimientos
for details in pokemon_details:
    for move in details['moves']:
        move_counts[move['move_name']] += 1

# Filtramos los movimientos que aparecen en al menos 3 Pokémon
common_moves = [move for move, count in move_counts.items() if count >= 3]

# Ahora, extraemos las características de estos movimientos comunes
moves_details = []

for move_name in common_moves:
    # Construimos la URL para la solicitud de la API del movimiento
    move_url = f'https://pokeapi.co/api/v2/move/{move_name}'

    # Realizamos la solicitud a la API
    move_response = requests.get(move_url)
    move_data = moves_response.json()

        # Extraemos la información de cada movimiento y la agregamos a la lista
    moves_details.append({
            'move_name': move_name,
            'accuracy': move_data.get('accuracy', 'N/A'),
            'power': move_data.get('power', 'N/A'),
            'pp': move_data['pp'],
            'type': move_data['type']['name']
        })
# Creamos un DataFrame con los detalles de los movimientos comunes
common_moves_df = pd.DataFrame(moves_details)

# Mostramos el DataFrame con los movimientos comunes
print(common_moves_df)

            move_name  accuracy  power  pp   type
0          mega-punch       100     50  20  grass
1           ice-punch       100     50  20  grass
2       thunder-punch       100     50  20  grass
3        swords-dance       100     50  20  grass
4           mega-kick       100     50  20  grass
..                ...       ...    ...  ..    ...
204  stomping-tantrum       100     50  20  grass
205      draco-meteor       100     50  20  grass
206         iron-head       100     50  20  grass
207    chilling-water       100     50  20  grass
208         power-gem       100     50  20  grass

[209 rows x 5 columns]


1.1 Salva los datos en un archivo csv

In [ ]:
common_moves_df.to_csv('pokemon_moves_3.csv',index=False)

2. Obten el nombre y caracteristicas todos los moviemientos que no se repiten en ningun otro pokemon, osea son movimientos unicos para un solo pokemon

In [ ]:
# Usaremos el mismo diccionario 'move_counts' para contar la frecuencia de cada movimiento

# Filtramos los movimientos que son únicos para un solo Pokémon
unique_moves = [move for move, count in move_counts.items() if count == 1]

# Extraemos las características de estos movimientos únicos
unique_moves_details = []

for move_name in unique_moves:
    # Construimos la URL para la solicitud de la API del movimiento
    move_url = f'https://pokeapi.co/api/v2/move/{move_name}'

    # Realizamos la solicitud a la API
    move_response = requests.get(move_url)

    # Verificamos que la respuesta sea exitosa
    if move_response.status_code == 200:
        move_data = move_response.json()

        # Extraemos la información de cada movimiento y la agregamos a la lista
        unique_moves_details.append({
            'move_name': move_name,
            'accuracy': move_data.get('accuracy', 'N/A'),
            'power': move_data.get('power', 'N/A'),
            'pp': move_data['pp'],
            'type': move_data['type']['name']
        })
    else:
        print(f"Error al obtener datos del movimiento {move_name} con status: {move_response.status_code}")

# Creamos un DataFrame con los detalles de los movimientos únicos
unique_moves_df = pd.DataFrame(unique_moves_details)

# Mostramos el DataFrame con los movimientos únicos
print(unique_moves_df)


       move_name  accuracy  power  pp     type
0      foresight       NaN    NaN  40   normal
1      bone-rush      90.0   25.0  10   ground
2    meteor-mash      90.0   90.0  10    steel
3    metal-sound      85.0    NaN  40    steel
4        copycat       NaN    NaN  20   normal
..           ...       ...    ...  ..      ...
102       pounce     100.0   50.0  20      bug
103      recover       NaN    NaN  10   normal
104  miracle-eye       NaN    NaN  40  psychic
105    psystrike     100.0  100.0  10  psychic
106  sand-attack     100.0    NaN  15   ground

[107 rows x 5 columns]


2.2 Salvalos a un csv

In [ ]:
unique_moves_df.to_csv('unrepeated_moves.csv',index=False)

# Manejo de tipos

Para cada tipo de pokemon (fire, water, grass, etc) que aparecen en los datos. Obten una serie de dataframes que contengan a los pokemones que son de cada tipo. Y colocalos en un diccionarion donde la llave (`key`) es el tipo

In [ ]:
# Suponiendo que 'pokemon_df' ya está definido y contiene una columna 'types' que es una lista de tipos para cada Pokémon

# Inicializamos un diccionario vacío para los DataFrames de tipos de Pokémon
df_tipos = {}

# Iteramos sobre cada Pokémon y sus tipos para llenar los DataFrames
for index, row in pokemon_df.iterrows():
    for tipo in row['types']:
        # Si el tipo aún no tiene un DataFrame, crea uno y añade el Pokémon
        if tipo not in df_tipos:
            df_tipos[tipo] = pd.DataFrame(columns=pokemon_df.columns)
        df_tipos[tipo] = df_tipos[tipo].append(row, ignore_index=True)


<ipython-input-12-e660b779932f>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tipos[tipo] = df_tipos[tipo].append(row, ignore_index=True)
<ipython-input-12-e660b779932f>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tipos[tipo] = df_tipos[tipo].append(row, ignore_index=True)
<ipython-input-12-e660b779932f>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tipos[tipo] = df_tipos[tipo].append(row, ignore_index=True)
<ipython-input-12-e660b779932f>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tipos[tipo] = df_tipos[tipo].append(row, ignore_index=True)
<ipython-input-12-e660b779932f>:12: FutureWarning: The frame.app

Despues guara ese diccionario en un json llamado `tipos.json`

In [ ]:
# Convertir cada DataFrame en un diccionario y guardarlo en un archivo JSON
pokemon_types_dict = {tipo: df.to_dict('records') for tipo, df in df_tipos.items()}

# Guardamos el diccionario en un archivo JSON
with open('pokemon_tipes.json', 'w') as f:
    json.dump(pokemon_types_dict, f)

# Seleccion de Pokemones

Ahora crea un DataFrame que para cada pokemon unicamente contenga los movimientos que son de su tipo (osea si es tipo fuego y voldaor solo usa movimientos de tipo volador y fuego). Y crea una tabla donde tengan los siguientes datos en formato `tidydata`:
1. `pokemon_id`
2. `pokemon_name`
3. `type`(s) cada pokemon tiene a 1 o dos tipos (no mas). Tienes que guardar ambos  
4. `name` (del movimiento)
5. `accuracy`
6. `type`
7. `power`
8. `pp`

In [ ]:
import pandas as pd

# Assuming pokemon_details is a list that contains each Pokémon's details
# We will use this to create our new DataFrame
# This list should be available from the previous steps

# Initialize an empty list to hold the data for the new DataFrame
filtered_moves = []

# Loop through each pokemon's details
for pokemon in pokemon_details:
    pokemon_id = pokemon['id']
    pokemon_name = pokemon['name']
    pokemon_types = pokemon['types']

    # Loop through each move of the pokemon
    for move in pokemon['moves']:
        move_name = move['move_name']
        move_url = move['move_url']
        # Fetch move details from the API
        move_response = requests.get(move_url)
        move_data = move_response.json()

        # Check if the move's type is one of the pokemon's types
        if move_data['type']['name'] in pokemon_types:
            filtered_moves.append({
                'pokemon_id': pokemon_id,
                'pokemon_name': pokemon_name,
                'type': ' & '.join(pokemon_types),  # Join the types with ' & ' if there are more than one
                'name': move_name,
                'accuracy': move_data.get('accuracy', 'N/A'),
                'move_type': move_data['type']['name'],
                'power': move_data.get('power', 'N/A'),
                'pp': move_data['pp']
            })

# Create the new DataFrame
moves_df = pd.DataFrame(filtered_moves)



Finalmente guarda los datos en `.parquet`

In [ ]:
# Save the DataFrame in Parquet format
moves_df.to_parquet('tabla_pokemon.parquet')

#print the DataFrame
print(moves_df)

     pokemon_id pokemon_name              type              name  accuracy  \
0           448      lucario  fighting & steel          low-kick     100.0   
1           448      lucario  fighting & steel           counter     100.0   
2           448      lucario  fighting & steel          reversal     100.0   
3           448      lucario  fighting & steel            detect       NaN   
4           448      lucario  fighting & steel         iron-tail      75.0   
..          ...          ...               ...               ...       ...   
265         445     garchomp   dragon & ground         dual-chop      90.0   
266         445     garchomp   dragon & ground  stomping-tantrum     100.0   
267         445     garchomp   dragon & ground    breaking-swipe     100.0   
268         445     garchomp   dragon & ground        scale-shot      90.0   
269         445     garchomp   dragon & ground   scorching-sands     100.0   

    move_type  power  pp  
0    fighting    NaN  20  
1    figh